In [ ]:
# Import libraries
import os
from openai import OpenAI
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [ ]:
class Website:
    """
    A utility class to represent a Website that we have scraped
    """
    url: str
    title: str
    text: str

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
# Initialise OpenAI client using Ollama
OLLAMA_BASE_URL = "http://localhost:11434/v1"
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='hohoho')

In [ ]:
# Define system prompt
system_prompt = """
You are a snarky and sceptical but very diligent assistant that analyses the \
content of website pages, and provides a short, snarky, humorous summary with \
probing or thought-provoking questions, ignoring text that might be navigation \
related. Respond in markdown. Do not wrap the markdown in a code block - \
respond just with the markdown.
"""

In [ ]:
# Define user prompt prefix
user_prompt_prefix = """
Here are the contents of a website page. \
Provide a short and engaging summary of the website page. \
If it includes multiple sections, then summarise each section into a separate \
bullet point.
"""

In [ ]:
# Define function to create system and user prefix prompt messages
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website.text}
    ]

In [ ]:
# Define function to summarise the contents of a website page using the latest free LLM model
def summarise(website):
    response = ollama.chat.completions.create(
        model = "gpt-oss:latest",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [ ]:
# Define function to display this nicely in the output, using markdown
def display_summary(url):
    summary = summarise(url)
    display(Markdown(summary))

In [ ]:
# Let's put it into action
analyse_this = Website("https://edwarddonner.com")
display_summary(analyse_this)